In [18]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import t

# Load data for Part 1
df = pd.read_csv('/Users/Sam/Downloads/midterm_partone.csv')

# Define dependent variable (Y) and independent variables (X)
Y = df['Stock Change'].values
X = df[['Inventory Turnover', 'Operating Profit', 'Interaction Effect', 
        'Current Ratio', 'Quick Ratio', 'Debt Asset Ratio']].values


In [19]:
# Define instrumental variables (Z) and the delta bias term (using [1, 1, 1] vector as specified)
Z = np.column_stack([np.ones(len(Y)), np.ones(len(Y)), np.ones(len(Y))])

# Define the GMM objective function with the delta term included
def gmm_with_delta(params, X, Y, Z):
    # Unpack the parameters: last element is delta, the rest are beta coefficients
    beta = params[:-1]
    delta = params[-1]
    
    # Calculate the predicted values using beta coefficients
    predicted = X @ beta
    # Moment condition with delta term incorporated
    moment_condition = Z.T @ (Y - predicted - delta * Z[:, 0])  # Applying delta to the [1,1,1] bias vector
    # Objective function: sum of squared moment conditions
    return np.sum(moment_condition**2)


In [20]:
# Initial parameter guesses (6 betas + 1 delta)
initial_params = np.zeros(X.shape[1] + 1)

# Optimize GMM objective function to find parameter estimates
result = minimize(gmm_with_delta, initial_params, args=(X, Y, Z))
estimated_params = result.x

# Extract beta estimates and delta estimate
beta_estimates = estimated_params[:-1]  # Beta coefficients
delta_estimate = estimated_params[-1]   # Delta (bias term)

# Display beta coefficients and delta
print("Estimated beta coefficients:", beta_estimates)
print("Estimated delta (bias term):", delta_estimate)


Estimated beta coefficients: [-4.87085945e-04 -5.05641126e-06 -1.30194482e-04 -3.89990646e-05
 -2.11497210e-05 -2.97101079e-06]
Estimated delta (bias term): -1.9773300651053175e-05


In [21]:
# Calculate residuals using estimated beta and delta
residuals = Y - (X @ beta_estimates + delta_estimate * Z[:, 0])

# Calculate standard error for delta
delta_standard_error = np.std(residuals) / np.sqrt(len(Y))

# Display standard error and t-statistic for delta
print("\nStatistical significance test for delta (bias term):")
print("Standard Error for delta:", delta_standard_error)

# Calculate t-statistic for delta
t_statistic_delta = delta_estimate / delta_standard_error
print("t-Statistic for delta:", t_statistic_delta)

# Interpretation for delta
print("\nInterpretation for delta:")
if abs(t_statistic_delta) > 1.96:
    print("Delta is statistically significant at the 5% confidence level, indicating bias may be justified.")
else:
    print("Delta is not statistically significant, suggesting the expert's claim of bias may not be justified.")



Statistical significance test for delta (bias term):
Standard Error for delta: 0.0119527770397252
t-Statistic for delta: -0.0016542850741159455

Interpretation for delta:
Delta is not statistically significant, suggesting the expert's claim of bias may not be justified.


In [22]:
# Define number of observations (n) and parameters (k)
n, k = X.shape

# Calculate standard errors for all coefficients
standard_errors = np.std(residuals) / np.sqrt(n) * np.ones_like(estimated_params)

# Calculate t-statistics and p-values for each parameter
t_stats = estimated_params / standard_errors
p_values = [2 * (1 - t.cdf(np.abs(t_stat), df=n - k - 1)) for t_stat in t_stats]


In [23]:
# Create summary table
summary_table = pd.DataFrame({
    'Coefficient': estimated_params,
    'Std. Error': standard_errors,
    't-Statistic': t_stats,
    'p-Value': p_values
}, index=['Beta' + str(i+1) for i in range(len(beta_estimates))] + ['Delta'])

# Display the summary table
print("\nGMM Summary Table:")
print(summary_table)

# Interpretation guidance
print("\nInterpretation for coefficients:")
print("If a coefficient's p-value is less than 0.05, it is statistically significant at the 5% level.")



GMM Summary Table:
       Coefficient  Std. Error  t-Statistic   p-Value
Beta1    -0.000487    0.011953    -0.040751  0.967499
Beta2    -0.000005    0.011953    -0.000423  0.999663
Beta3    -0.000130    0.011953    -0.010892  0.991311
Beta4    -0.000039    0.011953    -0.003263  0.997397
Beta5    -0.000021    0.011953    -0.001769  0.998588
Beta6    -0.000003    0.011953    -0.000249  0.999802
Delta    -0.000020    0.011953    -0.001654  0.998680

Interpretation for coefficients:
If a coefficient's p-value is less than 0.05, it is statistically significant at the 5% level.
